In [1]:
import pandas as pd
import altair as alt

In [2]:
# --- Load the datasets ---
wastew_df = pd.read_csv('D:\TOOLS\Respiratory_Virus_Wastewater_Surveillance.csv')
resp_df = pd.read_csv('D:\TOOLS\Provisional_Deaths_Due_to_Respiratory_Illnesses.csv')

In [3]:
print("Wastewater Dataset:")
print(wastew_df.head())
print("\nIllness Dataset:")
print(resp_df.head())

Wastewater Dataset:
   mmwr_week  week  week_start    week_end     season     pathogen  siteno  \
0     202533    33  08/10/2025  08/16/2025  2024-2025  Influenza A       5   
1     202533    33  08/10/2025  08/16/2025  2024-2025  Influenza A       4   
2     202533    33  08/10/2025  08/16/2025  2024-2025   SARS-CoV-2      10   
3     202509     9  02/23/2025  03/01/2025  2024-2025        PMMOV       8   
4     202533    33  08/10/2025  08/16/2025  2024-2025          RSV       4   

               sitename  year  target_wkavg_concentration  conc_toplot  \
0  Chicago Lawn/Ashburn  2025                0.000000e+00    -1.000000   
1      Austin/Montclare  2025                0.000000e+00    -1.000000   
2         Racine Ave PS  2025                1.391603e-02     4.143515   
3        Little Village  2025                1.179165e+09    15.071575   
4      Austin/Montclare  2025                0.000000e+00    -1.000000   

    wkavg_val wkavg_val_category                row_id  
0    0.00

In [4]:
# --- Harmonization and Preparation ---

# 1. Prepare resp_df: Renaming and maping COVID-19 to SARS-CoV-2
df_illness_prep = resp_df.rename(columns={'disease': 'pathogen'}).copy()
df_illness_prep['pathogen'] = df_illness_prep['pathogen'].replace('COVID-19', 'SARS-CoV-2')

In [5]:
# 2. Separate Influenza records for duplication (to match Wastewater's A/B split)
df_illness_influenza = df_illness_prep[df_illness_prep['pathogen'] == 'Influenza'].copy()

In [6]:
# Create copies for Influenza A and Influenza B
df_illness_influenza_a = df_illness_influenza.copy()
df_illness_influenza_a['pathogen'] = 'Influenza A'

df_illness_influenza_b = df_illness_influenza.copy()
df_illness_influenza_b['pathogen'] = 'Influenza B'

In [7]:
# Concatenate the new 'Influenza A/B' records with the rest of the illness data
df_illness_harmonized = df_illness_prep[df_illness_prep['pathogen'] != 'Influenza']
df_illness_harmonized = pd.concat([df_illness_harmonized, df_illness_influenza_a, df_illness_influenza_b], ignore_index=True)

In [8]:
# Drop the redundant date column
df_illness_harmonized = df_illness_harmonized.drop(columns=['current_week_ending'])

In [9]:
# Define merge keys
merge_keys = ['mmwr_week', 'week', 'week_start', 'week_end', 'season', 'pathogen']

In [10]:
# Perform an outer merge on the wastewater data (wastew_df) and harmonized illness data
df_merged = pd.merge(
    wastew_df,
    df_illness_harmonized,
    on=merge_keys,
    how='outer',
    suffixes=('_wastewater', '_illness')
)

In [11]:
# Save the final merged dataset
df_merged.to_csv('merged_respiratory_data.csv', index=False)

print("\nMerged Dataset Head:")
print(df_merged.head())
print(f"\nMerged dataset saved to 'merged_respiratory_data.csv' with {len(df_merged)} rows.")


Merged Dataset Head:
   mmwr_week  week  week_start    week_end     season     pathogen  siteno  \
0     201840    40  09/30/2018  10/06/2018  2018-2019  Influenza A     NaN   
1     201840    40  09/30/2018  10/06/2018  2018-2019  Influenza B     NaN   
2     201840    40  09/30/2018  10/06/2018  2018-2019          RSV     NaN   
3     201840    40  09/30/2018  10/06/2018  2018-2019   SARS-CoV-2     NaN   
4     201841    41  10/07/2018  10/13/2018  2018-2019  Influenza A     NaN   

  sitename  year  target_wkavg_concentration  conc_toplot  wkavg_val  \
0      NaN   NaN                         NaN          NaN        NaN   
1      NaN   NaN                         NaN          NaN        NaN   
2      NaN   NaN                         NaN          NaN        NaN   
3      NaN   NaN                         NaN          NaN        NaN   
4      NaN   NaN                         NaN          NaN        NaN   

  wkavg_val_category row_id_wastewater  percent    row_id_illness  
0       

In [12]:
# --- Correlation Analysis and Visualization (SARS-CoV-2) ---

# Calculate a city-wide average for wastewater concentration
df_analysis = df_merged.groupby(['mmwr_week', 'week_start', 'pathogen']).agg(
    wkavg_val=('wkavg_val', 'mean'),
    percent=('percent', 'first') # 'percent' is repeated across sites, so 'first' is fine
).reset_index()

In [13]:
# Focus on SARS-CoV-2 and drop rows with missing values for the calculation
df_sars = df_analysis[df_analysis['pathogen'] == 'SARS-CoV-2'].dropna(subset=['wkavg_val', 'percent'])

In [14]:
# Calculate the correlation
correlation = df_sars['wkavg_val'].corr(df_sars['percent'])

print(f"\nCorrelation between city-wide SARS-CoV-2 wastewater concentration and illness percentage: {correlation:.2f}")


Correlation between city-wide SARS-CoV-2 wastewater concentration and illness percentage: 0.21


In [15]:
# --- Plot the Data (Altair) ---

# Melt the DataFrame for proper Altair dual-axis plotting
df_sars_melt = df_sars.melt(
    id_vars=['week_start'],
    value_vars=['wkavg_val', 'percent'],
    var_name='Metric',
    value_name='Value'
)

# Convert week_start to datetime for proper sorting on the x-axis
df_sars_melt['week_start'] = pd.to_datetime(df_sars_melt['week_start'], format='%m/%d/%Y')

In [16]:
# Create a dual-axis chart for comparison
base = alt.Chart(df_sars_melt).encode(
    x=alt.X('week_start:T', title='Week Start Date'),
)

In [17]:
# Layer 1: Wastewater Concentration (wkavg_val)
line = base.mark_line(point=True).encode(
    y=alt.Y('Value:Q', axis=alt.Axis(title='Wastewater Conc. (wkavg_val)', titleColor='#5276A7')),
    color=alt.Color('Metric:N', legend=None),
    tooltip=['week_start', alt.Tooltip('Value', format='.2f')]
).transform_filter(
    alt.FieldEqualPredicate(field='Metric', equal='wkavg_val')
)

In [18]:
# Layer 2: Illness Percentage (percent)
line_percent = base.mark_line(point=True).encode(
    y=alt.Y('Value:Q', axis=alt.Axis(title='Illness Percentage', titleColor='#F18727')),
    color=alt.value('#F18727'),
    tooltip=['week_start', alt.Tooltip('Value', format='.2f')]
).transform_filter(
    alt.FieldEqualPredicate(field='Metric', equal='percent')
)

In [19]:
# Combine layers and resolve scales
chart = alt.layer(line, line_percent).resolve_scale(
    y='independent' # Use independent y-axes
).properties(
    title='SARS-CoV-2 Trends: Wastewater Concentration vs. Illness Percentage'
)

In [20]:
# Display the chart (Altair will render this automatically in a Jupyter environment)
chart
# chart.save('sars_cov2_correlation_chart.json') # Uncomment to save the chart as a JSON file

alt.LayerChart(...)

In [21]:
# --- SETUP: Load and Prepare Data for Time Lag Analysis ---

df_merged = pd.read_csv('merged_respiratory_data.csv')
df_merged['week_start'] = pd.to_datetime(df_merged['week_start'], format='%m/%d/%Y')

In [22]:
# Aggregate to city-wide data for analysis
df_analysis = df_merged.groupby(['mmwr_week', 'week_start', 'pathogen']).agg(
    wkavg_val=('wkavg_val', 'mean'),
    percent=('percent', 'first')
).reset_index()

In [23]:
# Filter for SARS-CoV-2 and sort by date
df_sars = df_analysis[df_analysis['pathogen'] == 'SARS-CoV-2'].sort_values('week_start').dropna(subset=['wkavg_val', 'percent']).copy()

# --- TIME-LAGGED CORRELATION ---

max_lag = 4
correlations = {}

In [24]:
# Calculate correlation for lags 0 through 4
for lag in range(max_lag + 1):
    # 'shift(-lag)' aligns current wastewater data with illness data 'lag' weeks ahead.
    df_sars[f'percent_lag_{lag}'] = df_sars['percent'].shift(-lag)

    # Calculate correlation between current wkavg_val and the future percent
    correlation_value = df_sars['wkavg_val'].corr(df_sars[f'percent_lag_{lag}'])
    correlations[lag] = correlation_value

print("\n--- Time-Lagged Correlations (SARS-CoV-2) ---")
for lag, corr in correlations.items():
    if lag == 0:
        print(f"Lag 0 (Same Week): {corr:.3f}")
    else:
        print(f"Lag {lag} Week(s) (Future): {corr:.3f}")


--- Time-Lagged Correlations (SARS-CoV-2) ---
Lag 0 (Same Week): 0.215
Lag 1 Week(s) (Future): 0.297
Lag 2 Week(s) (Future): 0.326
Lag 3 Week(s) (Future): 0.354
Lag 4 Week(s) (Future): 0.346


In [25]:
# --- VISUALIZATION OF CORRELATION MAGNITUDE ---

# Convert the correlation results to a DataFrame for plotting
df_corr = pd.DataFrame(list(correlations.items()), columns=['Lag (Weeks)', 'Correlation Coefficient'])

chart_lagged = alt.Chart(df_corr).mark_bar(color='#5276A7').encode(
    x=alt.X('Lag (Weeks):O', title='Lag (Weeks)'),
    y=alt.Y('Correlation Coefficient:Q', title='Correlation (r)'),
    tooltip=['Lag (Weeks)', alt.Tooltip('Correlation Coefficient', format='.3f')]
).properties(
    title='SARS-CoV-2 Correlation between Wastewater (Time t) and Illness (Time t + Lag)'
).interactive() # Allows zooming and panning

chart_lagged.save('sars_cov2_lagged_correlation.json')

In [26]:
# Display the resulting correlation plot (Altair will render this automatically)
chart_lagged

alt.Chart(...)

In [27]:
# ----------------------------------------------------------------------
# Comparison of Illness Percentage Trends by Pathogen (Time Series Plot)
# ----------------------------------------------------------------------

# Filter for clinically relevant pathogens for comparison
pathogens_to_compare = ['SARS-CoV-2', 'Influenza A', 'RSV']
df_pathogen_comp = df_analysis[df_analysis['pathogen'].isin(pathogens_to_compare)].dropna(subset=['percent'])

chart_pathogen = alt.Chart(df_pathogen_comp).mark_line(point=True).encode(
    x=alt.X('week_start:T', title='Week Start Date'),
    y=alt.Y('percent:Q', title='Illness Percentage', scale=alt.Scale(zero=False)),
    color=alt.Color('pathogen:N', title='Pathogen'),
    tooltip=['week_start', 'pathogen', alt.Tooltip('percent', format='.2f')]
).properties(
    title='Comparison of Illness Percentage Trends by Pathogen'
).interactive()

chart_pathogen # Display the chart

alt.Chart(...)

In [28]:
# ----------------------------------------------------------------------
# SARS-CoV-2: Wastewater Concentration vs. Illness Percentage (Scatter Plot)
# ----------------------------------------------------------------------

# Filter for SARS-CoV-2 data
df_sars = df_analysis[df_analysis['pathogen'] == 'SARS-CoV-2'].dropna(subset=['wkavg_val', 'percent'])

# Create a scatter plot of the two key metrics
scatter_plot = alt.Chart(df_sars).mark_point().encode(
    x=alt.X('wkavg_val:Q', title='Wastewater Concentration (wkavg_val)'),
    y=alt.Y('percent:Q', title='Illness Percentage'),
    tooltip=['week_start', alt.Tooltip('wkavg_val', format='.2f'), alt.Tooltip('percent', format='.2f')]
).properties(
    title='SARS-CoV-2: Wastewater Concentration vs. Illness Percentage'
)

In [29]:
# Add a linear regression line to visualize the correlation
fit_line = scatter_plot.transform_regression(
    'wkavg_val', 'percent'
).mark_line(color='red')

final_scatter = (scatter_plot + fit_line).interactive()

final_scatter # Display the chart

alt.LayerChart(...)